# YouTube video to audio

- Author: Pierre Guillou
- Blog post: [Video-to-Audio | A free Web APP to get mp3 audio file from any YouTube video](https://medium.com/@pierre_guillou/video-to-audio-a-notebook-and-web-app-to-get-mp3-audio-file-from-any-youtube-video-bb6a1c85390d)
- Date: 06/12/2023 (Revision: 13/05/2025)



In [ ]:
!pip install gradio>=3.50.2
!pip install yt-dlp>=2023.3.4
!pip install ffmpeg-python>=0.2.0

In [ ]:
import gradio as gr
import subprocess
import os
import tempfile
import re
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

# Install required packages if in Colab
if IN_COLAB:
    print("Running in Google Colab. Installing required packages...")
    !pip install -q gradio yt-dlp ffmpeg-python

def download_youtube_mp3(video_url):
    """
    Downloads the audio from a YouTube video in MP3 format using yt-dlp.

    Args:
        video_url (str): The YouTube video URL.

    Returns:
        tuple: (status, result) where status is a boolean indicating success and result is either the MP3 file path or an error message
    """
    try:
        # Create a temporary directory for the download
        temp_dir = tempfile.mkdtemp()

        # Options for yt-dlp
        output_filename_template = os.path.join(temp_dir, "%(title)s.%(ext)s")

        command = [
            "yt-dlp",
            "-x",  # Extract audio
            "--audio-format", "mp3",
            "--audio-quality", "0", # 0 for best VBR quality
            "-o", output_filename_template,
            video_url
        ]

        # Add cookies from browser if in Colab (helps with authentication)
        if IN_COLAB:
            command.insert(1, "--cookies-from-browser")
            command.insert(2, "chrome")

        # Execute the command
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stdout, stderr = process.communicate()

        if process.returncode == 0:
            # Find the MP3 file in the temporary directory
            for file in os.listdir(temp_dir):
                if file.endswith(".mp3"):
                    mp3_path = os.path.join(temp_dir, file)
                    return True, mp3_path
            return False, "Error: MP3 file not found after conversion"
        else:
            error_output = stderr.decode('utf-8', errors='ignore')

            # Check if it's a YouTube authentication error
            if "Sign in to confirm you're not a bot" in error_output or "cookies" in error_output:
                if IN_COLAB:
                    return False, "YouTube requires authentication for this video. Try running the following command first to authenticate:\n!yt-dlp --cookies-from-browser chrome -x --audio-format mp3 " + video_url
                else:
                    return False, "YouTube requires authentication for this video. This app cannot download videos that require login. Try another video or use the app locally."

            return False, f"Download error: {error_output}"

    except Exception as e:
        return False, f"An error occurred: {str(e)}"

# Gradio Interface
def youtube_to_mp3(youtube_url):
    if not youtube_url:
        return "Please enter a valid YouTube URL"

    # Check if the URL is valid
    if not re.match(r'^(https?://)?(www\.)?(youtube\.com|youtu\.?be)/.+$', youtube_url):
        return "Invalid YouTube URL. Please enter a URL in the format https://www.youtube.com/watch?v=..."

    success, result = download_youtube_mp3(youtube_url)

    if success and os.path.isfile(result):
        return result
    else:
        return result  # This is an error message

# Create the interface
demo = gr.Interface(
    fn=youtube_to_mp3,
    inputs=gr.Textbox(label="YouTube Video URL", placeholder="https://www.youtube.com/watch?v=..."),
    outputs=gr.Audio(label="Downloaded MP3") if IN_COLAB else gr.Textbox(label="Result"),
    title="YouTube to MP3 Converter",
    description="""Download the audio from a YouTube video in MP3 format.

**Important note**: """ + ("Running in Google Colab. This environment can handle most YouTube videos, including those requiring authentication." if IN_COLAB else "Due to YouTube restrictions, some videos requiring authentication cannot be downloaded in this Hugging Face Spaces environment. For full functionality, run this application locally or in Google Colab."),
    examples=[["https://www.youtube.com/watch?v=jNQXAC9IVRw"]]  # First YouTube video (Me at the zoo)
)

if __name__ == "__main__":
    # Launch parameters
    if IN_COLAB:
        demo.launch(debug=True)
    else:
        # Specific parameters for Hugging Face Spaces
        demo.launch(server_name="0.0.0.0",
                    server_port=7860,
                    share=False,
                    ssr_mode=False)


Running in Google Colab. Installing required packages...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://435fef6abed049176c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
